<a href="https://colab.research.google.com/github/akshatshah91/Game-AI/blob/master/GA_with_NN_for_GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gym
import torch
import torch.nn as nn
import copy

In [ ]:
generations    = 50
startingSize   = 100
bestPopulation = int(startingSize/2)
numTestRuns    = 3
mutationRate   = 0.05
input_size     = None
hidden_size    = None
hidden_size1   = None
hidden_size2   = None
output_size    = None
scoreCap       = None

In [ ]:
class NN1(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NN1, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [ ]:
class NN2(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(NN2, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, output_size)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out

In [ ]:
def generatePopulation(popSize, AIType):
  agents = []
  for x in range(popSize):
    tmp = None
    if AIType == 0:
      tmp = NN1(input_size, hidden_size, output_size)
    else:
      tmp = NN2(input_size, hidden_size1, hidden_size2, output_size)
    for param in tmp.parameters():
      param.requires_grad = False
    agents.append(tmp)
  return agents

In [ ]:
def selectBest(env, agents, numBest, iterations):
  rewards = []
  for x in range(len(agents)):
    rewards.append((runSimulation(env, agents[x], iterations), x))
  rewards.sort(reverse=True)
  selected = []
  totalScore = 0
  for x in range(numBest):
    selected.append(agents[rewards[x][1]])
    totalScore += rewards[x][0]
  return selected, totalScore/numBest, rewards[0][0]

In [ ]:
def runSimulation(env, agent, iterations):
  rewards = 0
  action = 0
  for x in range(iterations):
    s = env.reset()
    gameOver = False
    while not gameOver:
      output = agent(torch.from_numpy(s).type("torch.FloatTensor"))
      o = max(output)
      action = output.detach().numpy().tolist().index(o)
      s, r, gameOver, _ = env.step(action)
      rewards += r
  return rewards/iterations

In [ ]:
def mutate1(agent, mutationRate):
  child = copy.deepcopy(agent)
  for param in child.parameters():
    if len(param.shape) is 1:
      for x in range(param.shape[0]):
        param[x] += mutationRate * np.random.random()
    elif len(param.shape) is 2:
      for x in range(param.shape[0]):
        for y in range(param.shape[1]):
          param[x][y] += mutationRate * np.random.random()
  return child

In [ ]:
def mutate2(agent, mutationRate):
  child = copy.deepcopy(agent)
  for param in child.parameters():
    if len(param.shape) is 1:
      for x in range(param.shape[0]):
        if np.random.random() > 0.5:
          param[x] += mutationRate * np.random.random()
    elif len(param.shape) is 2:
      for x in range(param.shape[0]):
        for y in range(param.shape[1]):
          if np.random.random() > 0.5:
            param[x][y] += mutationRate * np.random.random()
  return child

In [ ]:
def config(env, x, y, z):
  if z == 0:
    mutationRate = 0.05
  else:
    mutationRate = 0.2

  totalScore = 0
  highScore = 0
  totalGens = 0
  for a in range(3):
    parents = generatePopulation(startingSize, x)
    averageScore = 0
    for gen in range(generations):
      selected, averageScore, score = selectBest(env, parents, bestPopulation, numTestRuns)

      mutated = []
      if y == 0:
        for s in selected:
          mutated.append(mutate1(s, mutationRate))
      else:
        for s in selected:
          mutated.append(mutate2(s, mutationRate))

      parents = selected + mutated
      totalGens += 1
      if score > highScore:
        highScore = score
      if averageScore == scoreCap:
        break
    totalScore += averageScore
    print("Round", a+1, "\nAverage Score:", averageScore, "\nHigh Score:", highScore)
  print("Total Average Score:", totalScore/3, "\n\n")

In [58]:
def allConfigurations(env):
  for x in range(2):
    for y in range(2):
      for z in range(2):
        if x == 0:
          print("Neural Network with", input_size, "-->", hidden_size, "-->", output_size, "and Relu")
        else:
          print("Neural Network with", input_size, "-->", hidden_size1, "-->", hidden_size2, "-->", output_size, "and Relu") 
        if y == 0:
          print("Mutate every weight")
        else:
          print("Mutate random amount and randomly selected weights")
        if z == 0:
          print("Mutation Rate: 0.05")
        else:
          print("Mutation Rate: 0.2")
        config(env, x, y, z)

In [ ]:
input_size     = 4
hidden_size    = 5
hidden_size1   = 10
hidden_size2   = 3
output_size    = 2
scoreCap       = 500
env = gym.make("CartPole-v1")
print("CartPole-v1")
allConfigurations(env)

CartPole-v1
Neural Network with 4 --> 5 --> 2 and Relu
Mutate every weight
Mutation Rate: 0.05
Round 1 
Average Score: 500.0 
High Score: 500.0
Round 2 
Average Score: 500.0 
High Score: 500.0
Round 3 
Average Score: 500.0 
High Score: 500.0
Total Average Score: 500.0 


Neural Network with 4 --> 5 --> 2 and Relu
Mutate every weight
Mutation Rate: 0.2
Round 1 
Average Score: 500.0 
High Score: 500.0
Round 2 
Average Score: 500.0 
High Score: 500.0
Round 3 
Average Score: 500.0 
High Score: 500.0
Total Average Score: 500.0 


Neural Network with 4 --> 5 --> 2 and Relu
Mutate random amount and randomly selected weights
Mutation Rate: 0.05
Round 1 
Average Score: 427.0733333333334 
High Score: 500.0
Round 2 
Average Score: 500.0 
High Score: 500.0
Round 3 
Average Score: 500.0 
High Score: 500.0
Total Average Score: 475.69111111111107 


Neural Network with 4 --> 5 --> 2 and Relu
Mutate random amount and randomly selected weights
Mutation Rate: 0.2
Round 1 
Average Score: 500.0 
High Scor

In [59]:
input_size     = 128
hidden_size    = 67
hidden_size1   = 87
hidden_size2   = 47
output_size    = 6
scoreCap       = 99999
generations = 10
startingSize = 10
bestPopulation = 5
env = gym.make("AirRaid-ram-v0")
print("AirRaid-ram-v0")
allConfigurations(env)

AirRaid-ram-v0
Neural Network with 128 --> 67 --> 6 and Relu
Mutate every weight
Mutation Rate: 0.05
Round 1 
Average Score: 563.3333333333333 
High Score: 1058.3333333333333
Round 2 
Average Score: 591.6666666666667 
High Score: 1750.0
Round 3 
Average Score: 485.0000000000001 
High Score: 1891.6666666666667
Total Average Score: 546.6666666666666 


Neural Network with 128 --> 67 --> 6 and Relu
Mutate every weight
Mutation Rate: 0.2
Round 1 
Average Score: 396.66666666666663 
High Score: 1408.3333333333333
Round 2 
Average Score: 346.6666666666667 
High Score: 1408.3333333333333
Round 3 
Average Score: 274.99999999999994 
High Score: 1408.3333333333333
Total Average Score: 339.4444444444444 


Neural Network with 128 --> 67 --> 6 and Relu
Mutate random amount and randomly selected weights
Mutation Rate: 0.05
Round 1 
Average Score: 1029.9999999999998 
High Score: 2666.6666666666665
Round 2 
Average Score: 701.6666666666666 
High Score: 2666.6666666666665
Round 3 
Average Score: 488.33